In [1]:
from typing import List, Dict, Optional
from dataclasses import dataclass
from prettytable import PrettyTable
import random
from datetime import datetime, timedelta

# Enhanced constants with documentation
POPULATION_SIZE: int = 20  # Size of population in each generation
NUM_ELITE_SCHEDULES: int = 2  # Number of best schedules to keep
TOURNAMENT_SELECTION_SIZE: int = 5  # Number of schedules in tournament selection
MUTATION_RATE: float = 0.1  # Probability of mutation
MAX_GENERATIONS: int = 100  # Maximum number of generations to run
TIME_SLOT_DURATION: int = 60  # Duration in minutes

@dataclass
class TimeSlot:
    start_time: datetime
    end_time: datetime

    def overlaps(self, other: 'TimeSlot') -> bool:
        return (self.start_time < other.end_time and 
                self.end_time > other.start_time)

    def __str__(self) -> str:
        return f"{self.start_time.strftime('%H:%M')}-{self.end_time.strftime('%H:%M')}"

class Schedule:
    def __init__(self):
        self.classes: List[Class] = []
        self.num_conflicts: int = 0
        self.fitness: float = -1
        self.data = Data()

    def calculate_fitness(self) -> float:
        self.num_conflicts = 0
        classes = self.get_classes()
        
        # More efficient conflict checking
        for i, class1 in enumerate(classes):
            for class2 in classes[i+1:]:
                if self._check_resource_conflict(class1, class2):
                    self.num_conflicts += 1

        # Penalize schedules with many conflicts more heavily
        self.fitness = 1 / (1.0 * self.num_conflicts + 1)
        return self.fitness

    def _check_resource_conflict(self, class1: 'Class', class2: 'Class') -> bool:
        if class1.time_slot.overlaps(class2.time_slot):
            # Check machine conflicts
            if class1.machine == class2.machine:
                return True
            
            # Check labour conflicts
            if class1.labour_name == class2.labour_name:
                return True
            
            # Check site conflicts
            if (class1.activity.name == class2.activity.name and 
                class1.construction_site != class2.construction_site):
                return True
        
        return False

@dataclass
class Activity:
    number: int
    name: str
    labour: str
    duration: int
    prerequisites: List[str] = None

    def __post_init__(self):
        if self.prerequisites is None:
            self.prerequisites = []

# Updated Data class with activity dependencies
class Data:
    def __init__(self):
        self._initialize_data()

    def _initialize_data(self):
        # Initialize with dependencies
        self.activities = [
            Activity(1, "Piling", "Piling Crew", 5, []),
            Activity(2, "Pile Cap", "Structure Crew", 3, ["Piling"]),
            Activity(3, "Pier", "Structure Crew", 4, ["Pile Cap"]),
            Activity(4, "Pier Cap", "Structure Crew", 3, ["Pier"]),
            Activity(5, "Girder Erection", "Structure Crew", 4, ["Pier Cap"])
        ]
        # ... rest of initialization code ...

def print_schedule_with_conflicts(schedule: Schedule) -> None:
    """Print schedule with detailed conflict information"""
    table = PrettyTable()
    table.field_names = ["Class #", "Site", "Activity", "Duration", "Machine", 
                        "Labour", "Working Hours", "Conflicts"]
    
    for class_item in schedule.get_classes():
        conflicts = []
        for other_class in schedule.get_classes():
            if class_item != other_class and schedule._check_resource_conflict(class_item, other_class):
                conflicts.append(f"Conflicts with {other_class.activity.name} at {other_class.construction_site}")
        
        table.add_row([
            class_item.class_num,
            class_item.construction_site,
            class_item.activity.name,
            class_item.activity.duration,
            class_item.machine.name,
            class_item.labour_name.name,
            class_item.time_slot,
            "\n".join(conflicts) if conflicts else "None"
        ])
    
    print(table)